In [177]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
import sys
sys.path.append("../")
from functools import reduce
import operator
import geopandas as gpd
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

In [102]:

from src.geocode import *
from src.getFromDict import getFromDict
from src.create_marker import create_marker

In [84]:
# Inicializar mongo
#sudo systemctl start mongod
#sudo systemctl  status mongod

In [85]:
from pymongo import MongoClient
conn = MongoClient("localhost:27017")
conn.list_database_names()
db = conn.get_database("ironhack")
db.list_collection_names()


['countries_small', 'books', 'restaurants', 'companies']

In [3]:
load_dotenv()

True

In [4]:
token1 = os.getenv("token1")
token2 = os.getenv("token2")

In [5]:
address = "Madrid"

In [6]:
madrid = geocode(address)

In [7]:
madrid

{'type': 'Point', 'coordinates': [-3.6793, 40.42955]}

In [66]:
 #Si geocode no funciona tenemos plan b
#if madrid["success"] == False:
 #   madrid = {'type': 'Point', 'coordinates': [-3.6793, 40.42955]}

In [78]:
madrid = {'type': 'Point', 'coordinates': [-3.6793, 40.42955]}

In [8]:
url_mother = 'https://api.foursquare.com/v2/venues/explore'


In [41]:
search ="4bf58dd8d48988d110941735"


In [10]:
print(madrid.get("coordinates")[0])

-3.6793


In [37]:
parametros = { "client_id" : token1,
          "client_secret" : token2,
          "v" : "20180323",
          "ll": f"{madrid.get('coordinates')[1]},{madrid.get('coordinates')[0]}",
          "query": f"{search}",
          "limit" : 100
}

In [45]:
resp = requests.get (url = url_mother, params = parametros)
data = json.loads(resp.text)

In [46]:

data.keys()

dict_keys(['meta', 'response'])

In [47]:
data

{'meta': {'code': 200, 'requestId': '5fb111923a1e82333e3a46ec'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 1869,
  'headerLocation': 'Lista',
  'headerFullLocation': 'Lista, Madrid',
  'headerLocationGranularity': 'neighborhood',
  'query': '4bf58dd8d48988d110941735',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': 40.47205489596985,
    'lng': -3.671258145249127},
   'sw': {'lat': 40.37340656429336, 'lng': -3.718670814180229}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d3b508bcc48224b8ad2454f',
       'name': 'Vinos De Bellota',
       'contact': {},
       'location': {'address': 'Calle de Antonio Acuña, 27',
        'crossStreet': 'Calle del Doctor Castelo

In [48]:
decoding_data = data.get("response")
decoding_data

{'suggestedFilters': {'header': 'Tap to show:',
  'filters': [{'name': 'Open now', 'key': 'openNow'}]},
 'suggestedRadius': 1869,
 'headerLocation': 'Lista',
 'headerFullLocation': 'Lista, Madrid',
 'headerLocationGranularity': 'neighborhood',
 'query': '4bf58dd8d48988d110941735',
 'totalResults': 23,
 'suggestedBounds': {'ne': {'lat': 40.47205489596985,
   'lng': -3.671258145249127},
  'sw': {'lat': 40.37340656429336, 'lng': -3.718670814180229}},
 'groups': [{'type': 'Recommended Places',
   'name': 'recommended',
   'items': [{'reasons': {'count': 0,
      'items': [{'summary': 'This spot is popular',
        'type': 'general',
        'reasonName': 'globalInteractionReason'}]},
     'venue': {'id': '4d3b508bcc48224b8ad2454f',
      'name': 'Vinos De Bellota',
      'contact': {},
      'location': {'address': 'Calle de Antonio Acuña, 27',
       'crossStreet': 'Calle del Doctor Castelo, 3',
       'lat': 40.42067595716258,
       'lng': -3.679058690556566,
       'labeledLatLngs': [

In [84]:
decoded = decoding_data.get("groups")
type(decoded)

list

In [85]:
type(decoded[0])

dict

In [83]:

for k,v in decoded[0].items():
    print(k,v,"\n")


type Recommended Places 

name recommended 

items [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4d3b508bcc48224b8ad2454f', 'name': 'Vinos De Bellota', 'contact': {}, 'location': {'address': 'Calle de Antonio Acuña, 27', 'crossStreet': 'Calle del Doctor Castelo, 3', 'lat': 40.42067595716258, 'lng': -3.679058690556566, 'labeledLatLngs': [{'label': 'display', 'lat': 40.42067595716258, 'lng': -3.679058690556566}], 'distance': 988, 'postalCode': '28009', 'cc': 'ES', 'city': 'Madrid', 'state': 'Madrid', 'country': 'España', 'formattedAddress': ['Calle de Antonio Acuña, 27 (Calle del Doctor Castelo, 3)', '28009 Madrid Madrid', 'España']}, 'categories': [{'id': '4bf58dd8d48988d150941735', 'name': 'Spanish Restaurant', 'pluralName': 'Spanish Restaurants', 'shortName': 'Spanish', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/spanish_', 'suffix': '.png'}, 'primary': True}],

In [88]:
restaurant = decoded[0].get("items")
type(restaurant)

list

In [92]:
type(restaurant[0])

dict

In [95]:
restaurant[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4d3b508bcc48224b8ad2454f',
  'name': 'Vinos De Bellota',
  'contact': {},
  'location': {'address': 'Calle de Antonio Acuña, 27',
   'crossStreet': 'Calle del Doctor Castelo, 3',
   'lat': 40.42067595716258,
   'lng': -3.679058690556566,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.42067595716258,
     'lng': -3.679058690556566}],
   'distance': 988,
   'postalCode': '28009',
   'cc': 'ES',
   'city': 'Madrid',
   'state': 'Madrid',
   'country': 'España',
   'formattedAddress': ['Calle de Antonio Acuña, 27 (Calle del Doctor Castelo, 3)',
    '28009 Madrid Madrid',
    'España']},
  'categories': [{'id': '4bf58dd8d48988d150941735',
    'name': 'Spanish Restaurant',
    'pluralName': 'Spanish Restaurants',
    'shortName': 'Spanish',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/spanish_',
   

In [94]:
restaurant[0].keys()

dict_keys(['reasons', 'venue', 'referralId'])

In [97]:
#saltos que damos dentro del diccionario para llegar al valor
map_name = ["venue","name"]
lat = ["venue", "location","lat"]
long = ["venue","location","lng"]

In [104]:
list_rest = []
for dict_elem in restaurant:
    list_test = {}
    list_test["name"] = getFromDict(dict_elem,map_name)
    list_test["latitud"] = getFromDict(dict_elem,lat)
    list_test["longitud"] = getFromDict(dict_elem,long)
    list_rest.append(list_test)

In [105]:
dataframefinal = pd.DataFrame(list_rest)
dataframefinal.head()

,name,latitud,longitud
0,Vinos De Bellota,40.420676,-3.679059
1,DDiego,40.434621,-3.676843
2,De María,40.425839,-3.697331
3,D'Mystic,40.423215,-3.698125
4,SANZ CABREJAS Bufete de Abogados penalista,40.435540,-3.686364


In [108]:
gdf = gpd.GeoDataFrame(dataframefinal, geometry=gpd.points_from_xy(dataframefinal.longitud, dataframefinal.latitud))
print(f'Tipo: {type(gdf)}')
gdf.head()

Tipo: <class 'geopandas.geodataframe.GeoDataFrame'>


,name,latitud,longitud,geometry
0,Vinos De Bellota,40.420676,-3.679059,POINT (-3.67906 40.42068)
1,DDiego,40.434621,-3.676843,POINT (-3.67684 40.43462)
2,De María,40.425839,-3.697331,POINT (-3.69733 40.42584)
3,D'Mystic,40.423215,-3.698125,POINT (-3.69813 40.42321)
4,SANZ CABREJAS Bufete de Abogados penalista,40.435540,-3.686364,POINT (-3.68636 40.43554)


###  Mapa

In [167]:
inicial_lat = 40.42955
inicial_long = -3.6793
type(inicial_lat)

float

In [168]:
madrid.get("coordinates")
print(madrid.get("coordinates"))
initial_long = madrid.get("coordinates")[0]
initial_lat = madrid.get("coordinates")[1]
initial_lat

[-3.6793, 40.42955]


40.42955

In [172]:
map_1 = folium.Map(location = [initial_lat,initial_long], zoom_start = 15)


In [178]:
create_marker(gdf,map_1)

In [179]:
map_1